In [1]:
import requests
from bs4 import BeautifulSoup as BS
from pprint import pprint
from pandas import json_normalize
import numpy as np
import pandas as pd
# '/vacancies/kassir'

In [2]:
class HH_scraper():
    
    def __init__(self, suffix):
        
        self.url = 'https://hh.ru'
        self.suffix = suffix
        self.page = self.url + self.suffix
        self.headers = {'User-Agent': 'Mozilla / 5.0 (Windows NT 10.0; Win64; x64) AppleWebKit / 537.36 (KHTML, like Gecko) Chrome / 81.0.4044.92 Safari / 537.36'}
        self.dom = None
        self.jobs_list = []
        
    
    def get_data(self):
        
        while(True):
            
            if self.page:
                response = requests.get(self.page, headers=self.headers)
                self.dom = BS(response.text, 'html.parser')

                try:
                    self.page = self.url + self.dom.find('a', {'class': 'bloko-button', 'data-qa': 'pager-next'}).get('href')
                    self.get_jobs()
                except:
                    self.page = None
            
            if not self.page:
                self.get_jobs()
                print('done')
                break
                
                
    def get_jobs(self):
        
        jobs = self.dom.find_all('div', {'class':'vacancy-serp-item vacancy-serp-item_redesigned'})
        
        for job in jobs:
            
            job_data = {}
            vacancy = job.find('a').text
            link = job.find('a').get('href')

            try:
                salary_split = job.find('span', {'class': 'bloko-header-section-3', 'data-qa': 'vacancy-serp__vacancy-compensation'}).text.split()
            except:
                salary_split = None

            if salary_split:
                currency = salary_split[-1]

                len_salary_split = len(salary_split)

                if len_salary_split == 6: # 10 000 - 20 000 руб. 
                    salary_min = int(f'{salary_split[0]}{salary_split[1]}')
                    salary_max = int(f'{salary_split[3]}{salary_split[4]}')
                if len_salary_split == 3: # от/до 800 руб.
                    if salary_split[0] == 'от':
                        salary_min = int(salary_split[1])
                        salary_max = None
                    else:
                        salary_max = int(salary_split[1])
                        salary_min = None
                if len_salary_split == 4: # (1. от/до 10 000 руб.) и (2. 800 - 900 руб.)
                    if salary_split[0] == 'от':
                        salary_min = int(f'{salary_split[1]}{salary_split[2]}')
                        salary_max = None
                    elif salary_split[0] == 'до':
                        salary_max = int(f'{salary_split[1]}{salary_split[2]}')
                        salary_min = None
                    else:
                        salary_min = int(salary_split[0])
                        salary_max = int(salary_split[2])
            else:
                salary_min = None
                salary_max = None
                currency = None
                
            job_data['salary_min'] = salary_min
            job_data['salary_max'] = salary_max
            job_data['currency'] = currency
            job_data['vacancy'] = vacancy
            job_data['link'] = link
            job_data['source'] = self.url
            
            self.jobs_list.append(job_data)

In [3]:
scrapper = HH_scraper('/vacancies/kassir')
scrapper.get_data()

done


In [4]:
df = json_normalize(scrapper.jobs_list)
df

,salary_min,salary_max,currency,vacancy,link,source
0,40000.0,45000.0,руб.,Кассир (м. Пушкинская),https://hh.ru/vacancy/50978992?query=%D0%BA%D0...,https://hh.ru
1,45000.0,80000.0,руб.,"Продавец-кассир в ""Павелецкую Плазу""",https://hh.ru/vacancy/52875977?query=%D0%BA%D0...,https://hh.ru
2,35000.0,45000.0,руб.,Продавец-кассир,https://hh.ru/vacancy/52897143?query=%D0%BA%D0...,https://hh.ru
3,42000.0,NaN,руб.,Продавец-кассир (ТРЦ Columbus),https://hh.ru/vacancy/43818771?query=%D0%BA%D0...,https://hh.ru
4,60000.0,NaN,руб.,"Продавец-кассир Karl Lagerfeld (ТЦ ""Мега Теплы...",https://hh.ru/vacancy/52153216?query=%D0%BA%D0...,https://hh.ru
...,...,...,...,...,...,...
1995,35000.0,NaN,руб.,"Кассир (Ул. Широкая, м. Медведково)",https://hh.ru/vacancy/51986798?query=%D0%BA%D0...,https://hh.ru
1996,40000.0,NaN,руб.,Продавец-кассир (ТЦ Гагаринский),https://hh.ru/vacancy/52533101?query=%D0%BA%D0...,https://hh.ru
1997,NaN,41800.0,руб.,Продавец-кассир (Семеновский),https://hh.ru/vacancy/50440635?query=%D0%BA%D0...,https://hh.ru
1998,42000.0,47000.0,руб.,Старший кассир-администратор (м.Динамо),https://hh.ru/vacancy/51399466?query=%D0%BA%D0...,https://hh.ru


In [5]:
df.to_csv('HH_kassir.csv')